In [1]:
!pip install yfinance timesfm==1.1.0


INFO: pip is looking at multiple versions of paxml to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of jax[cuda12] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax[cuda12] to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is still looking at multiple versions of paxml to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warning

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [3]:
!pip install yfinance
!pip install utilsforecast


In [4]:
import os
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
os.environ['JAX_PMAP_USE_TENSORSTORE'] = 'false'

In [5]:
import timesfm
import gc
import numpy as np
import pandas as pd
from timesfm import patched_decoder
from timesfm import data_loader
import yfinance as yf

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [6]:
import numpy as np
import tensorflow as tf
import random
import os

# Set a seed value
seed_value = 42

# Set Python random seed
random.seed(seed_value)

# Set NumPy random seed
np.random.seed(seed_value)

# Set TensorFlow random seed
tf.random.set_seed(seed_value)

# For reproducibility in other areas, like OS operations
os.environ['PYTHONHASHSEED'] = str(seed_value)


In [7]:
from tqdm import tqdm
import dataclasses
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [8]:
# Loading TimesFM pretrained checkpoint
tfm = timesfm.TimesFm(
    context_len=64,
    horizon_len=1,
    input_patch_len=32,
    output_patch_len=128,
    num_layers=20,
    model_dims=1280,
    backend="gpu",
)
tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/5.73k [00:00<?, ?B/s]

(…)oint_1100000/descriptor/descriptor.pbtxt:   0%|          | 0.00/499 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

...

(…)nts/checkpoint_1100000/metadata/metadata:   0%|          | 0.00/43.9k [00:00<?, ?B/s]

Constructing model weights.


Constructed model weights in 12.09 seconds.
Restoring checkpoint from /root/.cache/huggingface/hub/models--google--timesfm-1.0-200m/snapshots/8775f7531211ac864b739fe776b0b255c277e2be/checkpoints.


ERROR:absl:For checkpoint version > 1.0, we require users to provide
          `train_state_unpadded_shape_dtype_struct` during checkpoint
          saving/restoring, to avoid potential silent bugs when loading
          checkpoints to incompatible unpadded shapes of TrainState.


Restored checkpoint in 2.67 seconds.
Jitting decoding.
Jitted decoding in 25.98 seconds.


In [9]:
import pandas as pd
import yfinance as yf

# لیستی از نمادهای سهام معروف
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'JPM', 'V', 'NFLX']

# بارگذاری داده‌های بورسی روزانه برای هر سهام
all_data = []

for idx, ticker in enumerate(tickers, start=1):
    data = yf.download(ticker, start='1900-01-01', end='2024-01-01', interval='1d')
    if data.empty:
        print(f"Failed to download data for {ticker}")
        continue
    data.index = pd.to_datetime(data.index)
    data = data.dropna()
    data.reset_index(inplace=True)
    data.columns = data.columns.droplevel(1)
    data.rename(columns={'Date': 'date', 'Close': 'y'}, inplace=True)
    data['ticker'] = ticker  # Use ticker symbol instead of numerical ID

    final_cols = ["date", "y", "ticker"]
    data = data[final_cols]
    all_data.append(data)

# ترکیب داده‌های تمام سهام‌ها در یک DataFrame
combined_data = pd.concat(all_data)

# Pivot the DataFrame so that each stock is a column
combined_data_pivot = combined_data.pivot(index='date', columns='ticker', values='y')

# Reset index to make 'date' a column
combined_data_pivot.reset_index(inplace=True)

# Display the pivoted DataFrame
combined_data_pivot=combined_data_pivot.dropna()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [10]:
freq = 'D'
context_len = 64
pred_len = 1
batch_size = 32



In [11]:
data=combined_data_pivot

In [12]:
train_data = data[data['date'] < '2022-07-01']
val_data = data[(data['date'] >= '2022-07-01') & (data['date'] < '2023-01-01')]
test_data = data[data['date'] >= '2023-01-01']

In [13]:
from sklearn.preprocessing import StandardScaler

# Define the scalers dictionary
scalers = {}

def scale_data(data, scalers, fit=False):
    # Ensure index reset only once
    data_scaled = data.copy().reset_index(drop=True)

    # Define tickers as data columns if each ticker is a column


    for ticker in tickers:
        ticker_data = data[[ticker]]  # Ensure it's a DataFrame for scaling


        if fit:
            # Fit and save the scaler for each ticker
            scaler = StandardScaler()
            scaler.fit(ticker_data)
            scalers[ticker] = scaler
        else:
            scaler = scalers.get(ticker)
            if scaler is None:
                raise ValueError(f"No scaler found for ticker {ticker}. Ensure scalers are fitted.")

        # Scale the data for this ticker
        # data_scaled[ticker] = scaler.transform(ticker_data)

    return data_scaled

# Assuming train_data, val_data, and test_data are DataFrames with tickers as columns
# Fit scalers on training data
train_data_scaled = scale_data(train_data, scalers, fit=True)

# Scale validation and test data using the fitted scalers
val_data_scaled = scale_data(val_data, scalers, fit=False)
test_data_scaled = scale_data(test_data, scalers, fit=False)


In [14]:
data = pd.concat([train_data_scaled, val_data_scaled, test_data_scaled]).reset_index(drop=True)
print(f"Total data length after combining: {len(data)}")

Total data length after combining: 2923


In [15]:
data_path = 'stock_data.csv'
data.to_csv(data_path, index=False)

In [16]:
data

ticker,date,AAPL,AMZN,GOOGL,JPM,META,MSFT,NFLX,NVDA,TSLA,V
0,2012-05-18 00:00:00+00:00,18.942142,10.692500,15.025025,33.490002,38.230000,29.270000,9.994286,0.302000,1.837333,28.160000
1,2012-05-21 00:00:00+00:00,20.045713,10.905500,15.368118,32.509998,34.029999,29.750000,10.248571,0.307250,1.918000,29.055000
2,2012-05-22 00:00:00+00:00,19.891787,10.766500,15.035035,34.009998,31.000000,29.760000,9.672857,0.303500,2.053333,29.597500
3,2012-05-23 00:00:00+00:00,20.377144,10.864000,15.251752,34.259998,32.000000,29.110001,10.272857,0.311000,2.068000,29.792500
4,2012-05-24 00:00:00+00:00,20.190001,10.762000,15.106607,33.970001,33.029999,29.070000,10.038571,0.302750,2.018667,29.942499
...,...,...,...,...,...,...,...,...,...,...,...
2918,2023-12-22 00:00:00+00:00,193.600006,153.419998,141.490005,167.399994,353.390015,374.579987,486.760010,48.830002,252.539993,258.429993
2919,2023-12-26 00:00:00+00:00,193.050003,153.410004,141.520004,168.389999,354.829987,374.660004,491.190002,49.278999,256.609985,259.160004
2920,2023-12-27 00:00:00+00:00,193.149994,153.339996,140.369995,169.399994,357.829987,374.070007,491.790009,49.417000,261.440002,258.929993
2921,2023-12-28 00:00:00+00:00,193.580002,153.380005,140.229996,170.300003,358.320007,375.279999,490.510010,49.521999,253.179993,260.399994


In [17]:
ts_cols = [col for col in data.columns if col != "date"]
cat_cols=[col for col in ts_cols if col != "y"]

num_ts = len(ts_cols)
# Create TimeSeriesdata loader
dtl = data_loader.TimeSeriesdata(
      data_path=data_path,
      datetime_col="date",
      num_cov_cols=None,
      cat_cov_cols=None,
      ts_cols=ts_cols,
      train_range=[0, len(train_data)+64],
      val_range=[len(train_data)+64, len(train_data)+len(val_data)+64],
      test_range=[len(train_data)+len(val_data)+64, len(data)],
      hist_len=context_len,
      pred_len=pred_len,
      batch_size=1,  # Change as per requirement
      freq=freq,
      normalize=False,
      epoch_len=None,
      holiday=False,
      permute=False,
  )
train_batches = dtl.tf_dataset(mode="train", shift=1).batch(batch_size)
val_batches = dtl.tf_dataset(mode="val", shift=pred_len)
test_batches = dtl.tf_dataset(mode="test", shift=pred_len)

# MAE on the test split for the pretrained TimesFM model

In [18]:
for tbatch in tqdm(train_batches.as_numpy_iterator()):
    pass
print(tbatch[0].shape)
print(tbatch[1].shape)
print(tbatch[2].shape)

875it [00:09, 95.57it/s]

(27, 10, 64)
(27, 8, 64)
(27, 1, 64)


In [20]:
mse_losses = []
t=0
for batch in tqdm(test_batches.as_numpy_iterator()):
    t+=1
    if(t%10==1):
       past = batch[0]
       actuals = batch[3]

       _, forecasts = tfm.forecast(list(past), [0] * past.shape[0])
       forecasts = forecasts[:, 0 : actuals.shape[1], 5]

       mse_losses.append(np.square(forecasts - actuals).mean())

print(f"MSE: {np.mean(mse_losses)}")


1860it [00:04, 379.64it/s]


MSE: 5.423290252685547


# Fine-tuning the model

In [21]:
import jax
from jax import numpy as jnp
from praxis import pax_fiddle
from praxis import py_utils
from praxis import pytypes
from praxis import base_model
from praxis import optimizers
from praxis import schedules
from praxis import base_hyperparams
from praxis import base_layer
from paxml import tasks_lib
from paxml import trainer_lib
from paxml import checkpoints
from paxml import learners
from paxml import partitioning
from paxml import checkpoint_types

In [22]:
NestedMap = py_utils.NestedMap
WeightInit = base_layer.WeightInit
WeightHParams = base_layer.WeightHParams
InstantiableParams = py_utils.InstantiableParams
JTensor = pytypes.JTensor
NpTensor = pytypes.NpTensor
WeightedScalars = pytypes.WeightedScalars
instantiate = base_hyperparams.instantiate
LayerTpl = pax_fiddle.Config[base_layer.BaseLayer]
AuxLossStruct = base_layer.AuxLossStruct

AUX_LOSS = base_layer.AUX_LOSS
template_field = base_layer.template_field

# Standard prng key names
PARAMS = base_layer.PARAMS
RANDOM = base_layer.RANDOM

key = jax.random.PRNGKey(seed=1234)

In [23]:
model = pax_fiddle.Config(
    patched_decoder.PatchedDecoderFinetuneModel,
    name='patched_decoder_finetune',
    core_layer_tpl=tfm.model_p,
)

In [24]:

@pax_fiddle.auto_config
def build_learner() -> learners.Learner:
  return pax_fiddle.Config(
      learners.Learner,
      name='learner',
      loss_name='avg_qloss',
      optimizer=optimizers.Adam(
          epsilon=1e-7,
          clip_threshold=1e2,
          learning_rate=1e-2,
          lr_schedule=pax_fiddle.Config(
              schedules.Cosine,
              initial_value=1e-3,
              final_value=1e-4,
              total_steps=40000,
          ),
          ema_decay=0.9999,
      ),
      # Linear probing i.e we hold the transformer layers fixed.
      bprop_variable_exclusion=['.*/stacked_transformer_layer/.*'],
  )

In [25]:

task_p = tasks_lib.SingleTask(
    name='ts-learn',
    model=model,
    train=tasks_lib.SingleTask.Train(
        learner=build_learner(),
    ),
)

In [26]:
task_p.model.ici_mesh_shape = [1, 1, 1]
task_p.model.mesh_axis_names = ['replica', 'data', 'mdl']

DEVICES = np.array(jax.devices()).reshape([1, 1, 1])
MESH = jax.sharding.Mesh(DEVICES, ['replica', 'data', 'mdl'])

num_devices = jax.local_device_count()
print(f'num_devices: {num_devices}')
print(f'device kind: {jax.local_devices()[0].device_kind}')

num_devices: 1
device kind: Tesla T4


In [27]:
jax_task = task_p
key, init_key = jax.random.split(key)

# To correctly prepare a batch of data for model initialization (now that shape
# inference is merged), we take one devices*batch_size tensor tuple of data,
# slice out just one batch, then run the prepare_input_batch function over it.


def process_train_batch(batch):

    past_ts = batch[0].reshape(len(batch[2]) * num_ts, -1)
    actual_ts = batch[3].reshape(len(batch[2]) * num_ts, -1)
    return NestedMap(input_ts=past_ts, actual_ts=actual_ts)


def process_eval_batch(batch):
    past_ts = batch[0]
    actual_ts = batch[3]
    return NestedMap(input_ts=past_ts, actual_ts=actual_ts)


jax_model_states, _ = trainer_lib.initialize_model_state(
    jax_task,
    init_key,
    process_train_batch(tbatch),
    checkpoint_type=checkpoint_types.CheckpointType.GDA,
)

In [28]:
jax_model_states.mdl_vars['params']['core_layer'] = tfm._train_state.mdl_vars['params']
jax_vars = jax_model_states.mdl_vars
gc.collect()

0

In [29]:
jax_task = task_p


def train_step(states, prng_key, inputs):
  return trainer_lib.train_step_single_learner(
      jax_task, states, prng_key, inputs
  )


def eval_step(states, prng_key, inputs):
  states = states.to_eval_state()
  return trainer_lib.eval_step_single_learner(
      jax_task, states, prng_key, inputs
  )

key, train_key, eval_key = jax.random.split(key, 3)
train_prng_seed = jax.random.split(train_key, num=jax.local_device_count())
eval_prng_seed = jax.random.split(eval_key, num=jax.local_device_count())

p_train_step = jax.pmap(train_step, axis_name='batch')
p_eval_step = jax.pmap(eval_step, axis_name='batch')

In [30]:
replicated_jax_states = trainer_lib.replicate_model_state(jax_model_states)
replicated_jax_vars = replicated_jax_states.mdl_vars

In [31]:
best_eval_loss = 1e7
step_count = 0
patience = 0
NUM_EPOCHS = 100
PATIENCE = 5
TRAIN_STEPS_PER_EVAL = 1000
CHECKPOINT_DIR='/content'

In [32]:
def reshape_batch_for_pmap(batch, num_devices):
  def _reshape(input_tensor):
    bsize = input_tensor.shape[0]
    residual_shape = list(input_tensor.shape[1:])
    nbsize = bsize // num_devices
    return jnp.reshape(input_tensor, [num_devices, nbsize] + residual_shape)

  return jax.tree.map(_reshape, batch)

In [33]:
!rm -rf /content/checkpoint*


In [34]:
import pickle
save_count = 0
for epoch in range(NUM_EPOCHS):
    print(f"__________________Epoch: {epoch}__________________", flush=True)

    # Train phase
    train_its = train_batches.as_numpy_iterator()
    if patience >= PATIENCE:
        print("Early stopping.", flush=True)
        break

    train_losses = []  # Initialize train losses for the epoch
    for batch in train_its:
        if patience >= PATIENCE:
            print("Early stopping.", flush=True)
            break

        tbatch = process_train_batch(batch)
        tbatch = reshape_batch_for_pmap(tbatch, num_devices)
        replicated_jax_states, step_fun_out = p_train_step(
            replicated_jax_states, train_prng_seed, tbatch
        )
        train_losses.append(step_fun_out.loss[0])

        # Increment step count
        step_count += 1

    # Calculate and print average train loss for the epoch
    avg_train_loss = np.mean(train_losses)
    print(f"Train loss at end of epoch {epoch}: {avg_train_loss}", flush=True)

    # Eval phase at the end of the epoch
    print("Starting eval.", flush=True)
    val_its = val_batches.as_numpy_iterator()
    eval_losses = []  # Initialize eval losses for the epoch
    t=0
    for ev_batch in tqdm(val_its):
        t+=1
        if(t%10==1):

           ebatch = process_eval_batch(ev_batch)
           ebatch = reshape_batch_for_pmap(ebatch, num_devices)
           _, step_fun_out = p_eval_step(replicated_jax_states, eval_prng_seed, ebatch)
           eval_losses.append(step_fun_out.loss[0])

    # Calculate and print average eval loss for the epoch
    mean_loss = np.mean(eval_losses)
    print(f"Eval loss at end of epoch {epoch}: {mean_loss}", flush=True)

    # Check for checkpoint saving and patience logic
    if mean_loss < best_eval_loss or np.isnan(mean_loss):
        best_eval_loss = mean_loss

        jax_state_for_saving = py_utils.maybe_unreplicate_for_fully_replicated(
            replicated_jax_states
        )
        save_count += 1
        if save_count % 5 == 0:
            print("Unwaited save to Google Drive.")
            checkpoints.save_checkpoint(
                jax_state_for_saving, '/content/drive/MyDrive', overwrite=True,
            )
            with open(os.path.join( '/content/drive/MyDrive', 'train_state_info.pkl'), 'wb') as f:
               pickle.dump({'step_count': step_count, 'epoch': epoch}, f)
               print(f"Checkpoint saved at step {step_count}, epoch {epoch}")
        checkpoints.save_checkpoint(
            jax_state_for_saving, CHECKPOINT_DIR, overwrite=True
        )
        patience = 0  # Reset patience since we improved
        del jax_state_for_saving
        gc.collect()
    else:
        patience += 1
        print(f"patience: {patience}")

    # Early stopping check
    if patience >= PATIENCE:
        print("Early stopping.", flush=True)
        break


__________________Epoch: 0__________________
Train loss at end of epoch 0: 26.438549041748047
Starting eval.


1270it [00:13, 91.65it/s] 

Eval loss at end of epoch 0: 28.37788963317871



        shapes of train_state  as the unpadded shapes.


__________________Epoch: 1__________________
Train loss at end of epoch 1: 24.5494327545166
Starting eval.


...

patience: 3
__________________Epoch: 6__________________
Train loss at end of epoch 6: 20.08281707763672
Starting eval.


1270it [00:01, 741.84it/s]


Eval loss at end of epoch 6: 29.777788162231445
patience: 4
__________________Epoch: 7__________________
Train loss at end of epoch 7: 19.5490665435791
Starting eval.


1270it [00:01, 748.30it/s]

Eval loss at end of epoch 7: 28.89699363708496
patience: 5
Early stopping.


In [35]:
train_state = checkpoints.restore_checkpoint(jax_model_states, CHECKPOINT_DIR)
print(train_state.step)
tfm._train_state.mdl_vars['params'] = train_state.mdl_vars['params']['core_layer']
tfm.jit_decode()

ERROR:absl:For checkpoint version > 1.0, we require users to provide
          `train_state_unpadded_shape_dtype_struct` during checkpoint
          saving/restoring, to avoid potential silent bugs when loading
          checkpoints to incompatible unpadded shapes of TrainState.


2625
Jitting decoding.
Jitted decoding in 21.08 seconds.


In [36]:
mse_losses = []
t=0
for batch in tqdm(test_batches.as_numpy_iterator()):
    t+=1
    if(t%10==1):
       past = batch[0]
       actuals = batch[3]

       _, forecasts = tfm.forecast(list(past), [0] * past.shape[0])
       forecasts = forecasts[:, 0 : actuals.shape[1], 5]

       mse_losses.append(np.square(forecasts - actuals).mean())

print(f"MSE: {np.mean(mse_losses)}")


1860it [00:05, 354.64it/s]


MSE: 5.1173272132873535
